# Advanced Querying Mongo

Importing libraries and setting up connection

In [109]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [110]:
client.list_database_names()

['Crunchbase', 'admin', 'config', 'local', 'restaurants']

In [111]:
db = client.get_database("Crunchbase")

In [112]:
db.list_collection_names()

['companies']

In [113]:
collection = db.get_collection("companies")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [114]:
# Your Code
collection.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [115]:
# name field --> there is no such key in these documents
# we can try with 'description' or 'category_code'

In [116]:
filtro = {"name":'Babelgum'}
proj = {"category_code": 1}

list(collection.find(filtro, proj))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'category_code': 'games_video'}]

In [117]:
# there seems to be only one document named 'Babelgum' and it is a 'games & video' or a 'videogame' company.

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [118]:
# Your Code
filtro = {"number_of_employees": {"$gt": 5000}}
proj = {"_id": 0, "name": 1, "number_of_employees": 1}

list(collection.find(filtro, proj).limit(20).sort("number_of_employees",-1))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [119]:
# Your Code

# find({"$and": [cond1, cond2]})
proj = {"_id": 0, "name": 1, "founded_year": 1}

list(collection.find({"$and": [{"founded_year": {"$gte":2000}},{"founded_year": {"$lte":2005}}]}, proj).limit(11))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [120]:
# busco {'ipo': 0} en el filtro de mongo compass y no obtengo resultado. 
# Tampoco encuentro un 'ipo' haciendo find_one().keys()
# Aunque visualmente, lo encuentro dentro del campo 'milestones' y como un campo en sí con 'valuation_amount'

# No sé como hacer para obtener esta info de manera rápido

In [123]:
# Your Code

proj = {"_id": 0, "name": 1, "ipo": 1}
filtro = {"$and": [{"ipo.valuation_amount": {"$gt": 100000000}}, {"founded_year": {"$lte": 2010}}]}

list(collection.find(filtro, proj).limit(3))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [129]:
# Your Code
filtro = {"$and": [{"number_of_employees": {"$lte": 1000}}, {"founded_year": {"$lte": 2005}}]}
proj = {"_id": 0, "name": 1}

list(collection.find(filtro, proj).sort("number_of_employees",-1).limit(11))

[{'name': 'Telenav'},
 {'name': 'Sonus Networks'},
 {'name': 'FANUC Robotics America'},
 {'name': 'Omniture'},
 {'name': 'Akamai Technologies'},
 {'name': 'Antal International'},
 {'name': 'Riverbed Technology'},
 {'name': "OC'LIANE"},
 {'name': 'Gumtree'},
 {'name': 'SK Net Service Company Ltd'},
 {'name': 'Yodle'}]

### 6. All the companies that don't include the `partners` field.

In [145]:
# Your Code

proj = {"_id": 0, "name": 1}

list(collection.find({"partners":{"$exists": 'Null' }}, proj).limit(11))

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'},
 {'name': 'Omnidrive'},
 {'name': 'Postini'},
 {'name': 'Geni'},
 {'name': 'Flektor'},
 {'name': 'Fox Interactive Media'},
 {'name': 'Twitter'}]

In [146]:
# "$nin" creo que no se puede, lo he intentado también con False en vez de null pero me salía una lista vacía
# comprueblo en mongo compass y efectivamente no tienen ese field

### 7. All the companies that have a null type of value on the `category_code` field.

In [154]:
# Your Code

proj = {"_id": 0, "name": 1}

list(collection.find({"category_code": None}, proj).limit(11))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'Touch Clarity'},
 {'name': 'MMDAYS'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [161]:
# Your Code
filtro = {"$and": [{"number_of_employees": {"$gte": 100}}, {"number_of_employees": {"$lte": 1000}}]}
proj = {"_id": 0, "name": 1, "number_of_employees": 1}

list(collection.find(filtro, proj).limit(11))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106}]

### 9. Order all the companies by their IPO price in a descending order.

In [163]:
# Your Code

proj = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}
filtro = {"ipo.valuation_amount": {"$ne": None}}

list(collection.find(filtro, proj).sort("ipo.valuation_amount", -1).limit(11))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [164]:
# Your Code
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filtro = {"number_of_employees": {"$exists": True}}

list(collection.find(filtro, proj).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [167]:
# Your Code
# second semester of the year --> en 'founded_month' --> pillamos solo los últimos seis meses (semestre)

proj = {"_id": 0, "name": 1, "founded_month": 1}
filtro = {"founded_month": {"$in": [7, 8, 9, 10, 11, 12]}}

# lo podría hacer también con un $and de $gt 12 y $lt 7

list(collection.find(filtro, proj).sort("founded_month", -1).limit(1000));

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [175]:
# Your Code
filtro = {"$and": [{"acquisition.price_amount": {"$gte": 10_000_00}}, {"founded_year": {"$lte": 2000}}]}
proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}

list(collection.find(filtro, proj).limit(11))

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld', 'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets', 'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems', 'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Sabre', 'acquisition': {'price_amount': 4300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [174]:
# Your Code
filtro = {"acquisition.acquired_year": {"$gte": 2010}}
proj = {"_id": 0, "name": 1, "acquisition": 1}

list(collection.find(filtro, proj).sort("acquisition.price_amount", -1).limit(2))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [178]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_year": 1}
filtro = {"founded_year": {"$exists": True}}

list(collection.find(filtro, proj).sort("founded_year", -1).limit(10))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [180]:
# Your Code
proj = {"_id": 0, "name": 1, "founded_day": 1}
filtro = {"founded_day": {"$in": [1, 2, 3, 4, 5, 6, 7]}}

list(collection.find(filtro, proj).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'Vitrue', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [182]:
# Your Code

filtro = {"$and": [{"category_code": "web"}, {"number_of_employees": {"$gte": 4000}}]}
proj = {"_id": 0, "name": 1, "number_of_employees": 1}

list(collection.find(filtro, proj).sort("number_of_employees").limit(11))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [183]:
# Your Code
filtro = {"$and": [{"acquisition.price_currency_code": "EUR"}, {"acquisition.price_amount": {"$gte": 10_000_000}}]}
proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}

list(collection.find(filtro, proj).limit(11))

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}},
 {'name': 'Tuenti Technologies', 'acquisition': {'price_amount': 70000000}},
 {'name': 'BioMed Central', 'acquisition': {'price_amount': 43400000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [186]:
# Your Code

# first trimester of the year --> months 1, 2 and 3

filtro = {"acquisition.acquired_month": {"$in":[1, 2, 3]}}
proj = {"_id": 0, "name": 1, "acquisition": 1}

list(collection.find(filtro, proj).limit(2))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [187]:
# Your Code
filtro = {"$and": [{"acquisition.acquired_year": {"$gte":2011}}, {"founded_year": {"$in": [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]}}]}
proj = {"_id": 0, "name": 1, "acquisition.acquired_year": 1}

list(collection.find(filtro, proj).sort("acquisition.acquired_year").limit(11))

[{'name': 'Instructables', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks', 'acquisition': {'acquired_year': 2011}},
 {'name': 'hi5', 'acquisition': {'acquired_year': 2011}},
 {'name': 'delicious', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Dogster', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Flixster', 'acquisition': {'acquired_year': 2011}},
 {'name': 'TimeBridge', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Flock', 'acquisition': {'acquired_year': 2011}},
 {'name': 'IceRocket', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Zvents', 'acquisition': {'acquired_year': 2011}},
 {'name': 'Kyte', 'acquisition': {'acquired_year': 2011}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code
